In [1]:
!pip install ensemble_boxes

  Using cached numba-0.53.1-cp37-cp37m-manylinux2014_x86_64.whl (3.4 MB)
  Using cached llvmlite-0.36.0-cp37-cp37m-manylinux2010_x86_64.whl (25.3 MB)


In [1]:
import pandas as pd
from ensemble_boxes import *
import numpy as np
from pycocotools.coco import COCO

In [2]:
# ensemble csv files
submission_files = ['DetectoRS_ver1_fold0.csv', 'DetectoRS_ver1_fold1.csv', 'DetectoRS_ver1_fold2.csv', 'DetectoRS_ver1_fold3.csv', 'DetectoRS_ver1_fold4.csv', 'VFNet_ver1_fold0.csv', 'VFNet_ver1_fold2.csv', 'VFNet_ver1_fold3.csv', 'VFNet_ver1_fold4.csv']
submission_df = [pd.read_csv(file) for file in submission_files]

In [3]:
image_ids = submission_df[0]['image_id'].tolist()

In [4]:
annotation = '/opt/ml/input/data/test.json'
coco = COCO(annotation)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [5]:
prediction_strings = []
file_names = []
iou_thr = 0.4

for i, image_id in enumerate(image_ids):
    prediction_string = ''
    boxes_list = []
    scores_list = []
    labels_list = []
    image_info = coco.loadImgs(i)[0]
    for df in submission_df:
        predict_string = df[df['image_id'] == image_id]['PredictionString'].tolist()[0]
        predict_list = str(predict_string).split()
        if len(predict_list)==0 or len(predict_list)==1:
            continue
        predict_list = np.reshape(predict_list, (-1, 6))
        box_list = []
        for box in predict_list[:, 2:6].tolist():
            box[0] = float(box[0]) / image_info['width']
            box[1] = float(box[1]) / image_info['height']
            box[2] = float(box[2]) / image_info['width']
            box[3] = float(box[3]) / image_info['height']
            box_list.append(box)
        boxes_list.append(box_list)
        scores_list.append(list(map(float, predict_list[:, 1].tolist())))
        labels_list.append(list(map(int, predict_list[:, 0].tolist())))
    
    if len(boxes_list):
        boxes, scores, labels = nms(boxes_list, scores_list, labels_list, iou_thr=iou_thr)
        for box, score, label in zip(boxes, scores, labels):
            prediction_string += str(label) + ' ' + str(score) + ' ' + str(box[0] * image_info['width']) + ' ' + str(box[1] * image_info['height']) + ' ' + str(box[2] * image_info['width']) + ' ' + str(box[3] * image_info['height']) + ' '
    
    prediction_strings.append(prediction_string)
    file_names.append(image_id)

Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!


In [6]:
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv('submission_ensemble.csv')

submission.head()

,PredictionString,image_id
0,0 0.13577287 454.10657 236.71783 511.2198 293....,batch_01_vt/0021.jpg
1,0 0.17097556591033936 1.2692745923995972 242.4...,batch_01_vt/0028.jpg
2,0 0.12497574836015701 234.18716430664062 459.9...,batch_01_vt/0031.jpg
3,0 0.13087732 290.7732 152.64912 363.24817 227....,batch_01_vt/0032.jpg
4,0 0.29251698 144.82532 298.2073 289.00824 418....,batch_01_vt/0070.jpg
